In [ ]:
pip install plotly panel

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [ ]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('combined_transactions_categories.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

In [ ]:
import pandas as pd
import plotly.express as px

def make_pie_chart(df, year, label):
    # Filter the dataset for the specified expense/income and year
    sub_df = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]

    # Remove NaN values from the 'Category' column
    sub_df = sub_df.dropna(subset=['Category'])

    # Check if the filtered dataframe is empty after removing NaN values
    if sub_df.empty:
        raise ValueError(f"No data available for {label} in year {year}")

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount', names='Category', color_discrete_sequence=color_scale)
    pie_fig.update_traces(textposition='inside', direction='clockwise', hole=0.3, textinfo="label+percent")

    # Ensure Amount (EUR) is numeric
    sub_df['Amount'] = pd.to_numeric(sub_df['Amount'], errors='coerce')
    total_expense = sub_df['Amount'].sum()

    if pd.isna(total_expense):
        raise ValueError("Total expense calculation returned NaN")

    total_text = "€ " + str(round(total_expense))

    pie_fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode='hide',
        title=dict(text=label + " Breakdown " + str(year)),
        annotations=[
            dict(
                text=total_text,
                x=0.5, y=0.5, font_size=12,
                showarrow=False
            )
        ]
    )

    return pie_fig

# Example usage
try:
    expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')
    expense_pie_fig_2024.show()  # Use .show() to display the figure
except Exception as e:
    print(f"Error: {e}")


In [ ]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount', text_auto='.2s', title=label+" per month", color='Amount', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [ ]:
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')
expense_monthly_2024

In [ ]:
# Creating the charts
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')

# Create tabs
tabs = pn.Tabs(
    ('2024', pn.Column(pn.Row(expense_monthly_2024, expense_pie_fig_2024)))
)

# Display the tabs
tabs.show()